# Courtesy and Insights

https://blog.roboflow.com/keypoint-detection-on-roboflow/

James Gallagher. (Jan 5, 2024). Launch: Label, Train, Deploy Support for Keypoint Detection Models in Roboflow. Roboflow Blog: https://blog.roboflow.com/keypoint-detection-on-roboflow/

https://github.com/roboflow/notebooks/blob/main/notebooks/train-yolov8-keypoint.ipynb

# Step 1. Generate video with eyes wide open

Use webcam





# Step 2. Convert video frames to jpeg images

In [ ]:
import cv2
import os
from google.colab import drive


# Mount Google Drive
drive.mount('/content/drive')

# Path to the video file
video_path = '/content/drive/My Drive/Colab Notebooks/{input}.mp4'

# Directory to save the frames locally in Colab
save_dir = '/content/car_frames'
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

# Capture the video from the file
cap = cv2.VideoCapture(video_path)

current_frame = 0
while True:
    # Read the next frame from the video
    ret, frame = cap.read()

    if not ret:
        print("Finished reading the video.")
        break

    # Save the frame as a JPEG file
    save_path = os.path.join(save_dir, f"frame_{current_frame:04d}.jpg")
    cv2.imwrite(save_path, frame)
    print(f"Saved: {save_path}")

    current_frame += 1

cap.release()

print("All frames are extracted and saved locally.")


# Step 3. Upload images to Roboflow.

Do manual annotation

Add Labels

Add Bounding boxes.


https://universe.roboflow.com/test-4h9f7/sleep-detection-zw30k

Split Images to train, test, valid datasets

# Step 4. Install packages

In [ ]:
import os
HOME = os.getcwd()
print(HOME)

/content


In [ ]:
!pip install ultralytics==8.0.196 -q

import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.0.196 🚀 Python-3.10.12 torch-2.3.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 29.7/78.2 GB disk)


In [ ]:
from ultralytics import YOLO

from IPython.display import display, Image

# Step 5. Download Dataset from roboflow

In [ ]:
import getpass
import os
YOUR_API_KEY = getpass.getpass()

··········


In [ ]:

!mkdir {HOME}/datasets
%cd {HOME}/datasets

!pip install roboflow --quiet


/content/datasets
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.7/178.7 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 3.3 MB/s eta 0:00:00


In [ ]:
import requests

#YOUR_API_KEY = "YOUR_ACTUAL_API_KEY"  # Replace with your actual API key

url = "https://api.roboflow.com/"
headers = {"Authorization": f"Bearer {YOUR_API_KEY}"}

response = requests.get(url, headers=headers)

if response.status_code == 200:
    workspaces = response.json()
    print(workspaces)
else:
    print(f"Error: {response.status_code}")
    print(response.text)

{'welcome': 'Welcome to the Roboflow API.', 'instructions': 'You are successfully authenticated.', 'docs': 'https://docs.roboflow.com', 'workspace': 'test-4h9f7'}


In [ ]:
from roboflow import Roboflow

rf = Roboflow(api_key=YOUR_API_KEY)

project = rf.workspace("test-4h9f7").project("sleep-detection-zw30k")

version = project.version(1)

dataset = version.download("yolov8")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to sleep-detection-1 in yolov8:: 100%|██████████| 218/218 [00:00<00:00, 4024.38it/s]


In [ ]:
!ls -ltr /content/datasets/

total 4
drwxr-xr-x 5 root root 4096 Jul 29 21:36 sleep-detection-1


# Step 6. Train ultralytics YOLO model

In [ ]:
from ultralytics import YOLO

model = YOLO('yolov8m.pt')

results = model.train(data=f"{dataset.location}/data.yaml", epochs=40, imgsz=640)

100%|██████████| 49.7M/49.7M [00:00<00:00, 229MB/s]
New https://pypi.org/project/ultralytics/8.2.68 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.196 🚀 Python-3.10.12 torch-2.3.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8m.pt, data=/content/datasets/sleep-detection-1/data.yaml, epochs=40, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=None, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, st

In [ ]:
!ls -ltr runs/detect/train/weights/best.pt

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Step 7. Run Prediction on test images

In [ ]:

# Step 2: Import the necessary modules
import cv2
from ultralytics import YOLO
import urllib.request
from google.colab.patches import cv2_imshow
import os
import time

# Step 3: Load the pre-trained YOLOv8 model
model = YOLO('runs/detect/train/weights/best.pt')  # Load a pre-trained YOLOv8 model



# Function to detect objects in each frame of a video and draw bounding boxes
def detect_and_save_bounding_boxes(video_path, output_path, duration=25):
    # Open the video file
    cap = cv2.VideoCapture(video_path)

    if not cap.isOpened():
        print(f"Error opening video file: {video_path}")
        return

    # Get video properties
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Use 'mp4v' codec for compatibility
    out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

    # Initialize the start time
    start_time = time.time()
    frame_count = 0

    # Tracking variable to detect presence of certain classes
    eyes_detected = False

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Calculate elapsed time
        elapsed_time = time.time() - start_time
        if elapsed_time > duration:
            break

        # Run the model on the frame
        results = model(frame)
        #print ('results',results)

        # Check for person detections
        eyes_detected = False  # Reset the flag for each frame
        for result in results:
            for detection in result.boxes:
                class_id = int(detection.cls[0])  # Get the class id
                class_name = model.names[class_id]  # Get the class name

                # Get bounding box coordinates
                x1, y1, x2, y2 = map(int, detection.xyxy[0])
                # Draw bounding box
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                # Add label
                label = f'{class_name}'
                cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

                #print ('class_name',class_name)

                # If a person is detected
                if class_name == 'eye':
                    eyes_detected = True
                #print ('eyes_detected',eyes_detected)

        # Add text based on detection
        if eyes_detected:
            cv2.putText(frame, 'Awake', (50, 350), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 255, 0), 2)
        else:
            cv2.putText(frame, 'Sleeping', (50, 350), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 255), 2)

        # Write the frame with bounding boxes to the output video
        out.write(frame)
        frame_count += 1

    print(f"Processed {frame_count} frames.")


    # Release resources
    cap.release()
    out.release()
    cv2.destroyAllWindows()

# Path to the video file in Google Drive
video_path = "/content/drive/My Drive/Colab Notebooks/sleep.mp4"
output_path = "sleep_output_video.mp4"

# Detect and save bounding boxes for all detected objects in the video
detect_and_save_bounding_boxes(video_path, output_path, duration=50)




0: 384x640 2 eyes, 26.2ms
Speed: 2.6ms preprocess, 26.2ms inference, 2.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 eyes, 25.3ms
Speed: 3.0ms preprocess, 25.3ms inference, 2.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 eyes, 25.1ms
Speed: 3.2ms preprocess, 25.1ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 eyes, 24.0ms
Speed: 2.9ms preprocess, 24.0ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 eyes, 24.0ms
Speed: 2.5ms preprocess, 24.0ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 eyes, 24.0ms
Speed: 2.9ms preprocess, 24.0ms inference, 1.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 eyes, 27.6ms
Speed: 3.1ms preprocess, 27.6ms inference, 3.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 eyes, 20.9ms
Speed: 4.7ms preprocess, 20.9ms inference, 2.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x

Processed 797 frames.


In [ ]:
from google.colab import files

# Download the processed video file
files.download(output_path)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Sample Output 

![Sample Output](awake.png)

![Sample Output](sleeping.png)

# You can watch the full video here

https://github.com/AIBotTeachesAI/Sleep_Detection_Eye_Tracking_Custom_Fine_Tuning_YOLO/blob/main/README.md